In [1]:
from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langserve import add_routes
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

In [18]:
load_dotenv()

os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

groq_api_key = os.getenv("GROQ_API_KEY")

In [19]:
model=ChatGroq(model="Llama3-8b-8192",groq_api_key=os.getenv("GROQ_API_KEY"))

In [20]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="My name is subhasish and I am a software engineer")])

AIMessage(content="Nice to meet you, Subhasish! As a software engineer, I'm sure you're passionate about building innovative solutions and solving complex problems. What kind of projects have you been working on lately? Are you interested in any specific areas of software engineering, such as AI, machine learning, or cybersecurity?", response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 22, 'total_tokens': 84, 'completion_time': 0.051666667, 'prompt_time': 0.00379196, 'queue_time': 0.010855339, 'total_time': 0.055458627}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-51d512b1-bd6c-4670-a0b4-cc3c46352122-0', usage_metadata={'input_tokens': 22, 'output_tokens': 62, 'total_tokens': 84})

In [21]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="My name is subhasish and I am a software engineer"),
        AIMessage(content="Nice to meet you, Subhasish! Welcome to our conversation. As a software engineer, you must be passionate about coding and building innovative solutions. What kind of projects have you worked on in the past, and what are you currently focusing on?"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="I remember! Your name is Subhasish, and you're a software engineer!", response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 93, 'total_tokens': 111, 'completion_time': 0.015, 'prompt_time': 0.01040647, 'queue_time': 0.011888537, 'total_time': 0.02540647}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_af05557ca2', 'finish_reason': 'stop', 'logprobs': None}, id='run-4fe66b4f-aabe-4fbc-87fc-58feaba6326e-0', usage_metadata={'input_tokens': 93, 'output_tokens': 18, 'total_tokens': 111})

In [22]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import  BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if  session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]
        
with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [23]:
config={"configurable":{"session_id":"Chat-1"}}

In [24]:
with_message_history.invoke([HumanMessage(content="My name is subhasish and I am a software engineer")],
                            config=config)

AIMessage(content='Nice to meet you, Subhasish! As a software engineer, you must be working on exciting projects and solving complex problems. What kind of projects do you usually work on? Are you working on a specific technology stack or language?', response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 22, 'total_tokens': 70, 'completion_time': 0.04, 'prompt_time': 0.002355018, 'queue_time': 0.016464361, 'total_time': 0.042355018}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_af05557ca2', 'finish_reason': 'stop', 'logprobs': None}, id='run-ce713549-ddb3-4087-9fec-815f0b0b222d-0', usage_metadata={'input_tokens': 22, 'output_tokens': 48, 'total_tokens': 70})

In [25]:
config1={"configurable":{"session_id":"Chat-1"}}
response=with_message_history.invoke([HumanMessage(content="What is my Name?")],
                            config=config1)
response.content

'Your name is Subhasish.'

In [26]:
from langchain_core.prompts import  ChatPromptTemplate,MessagesPlaceholder

prompt=ChatPromptTemplate.from_messages([("system","You are a helpful assistant. Answer all the question to the best of your ability"),
                                         MessagesPlaceholder(variable_name="messages")])

chain=prompt|model

In [27]:
chain.invoke({"messages":[HumanMessage(content="My name is subhasish and I am a software engineer")]})

AIMessage(content="Nice to meet you, Subhasish! It's great to know that you're a software engineer. What kind of software development do you work on? Are you more interested in front-end, back-end, or full-stack development?", response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 43, 'total_tokens': 91, 'completion_time': 0.04, 'prompt_time': 0.005274444, 'queue_time': 0.008013114, 'total_time': 0.045274444}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_873a560973', 'finish_reason': 'stop', 'logprobs': None}, id='run-4273d145-9d04-4d67-aae7-4034a367014a-0', usage_metadata={'input_tokens': 43, 'output_tokens': 48, 'total_tokens': 91})

In [28]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [29]:
config2={"configurable":{"session_id":"Chat-1"}}
with_message_history.invoke([HumanMessage(content="What is my Name?")],config=config2)

AIMessage(content='Your name is Subhasish.', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 127, 'total_tokens': 135, 'completion_time': 0.006666667, 'prompt_time': 0.049170251, 'queue_time': 0.7968561409999999, 'total_time': 0.055836918}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_873a560973', 'finish_reason': 'stop', 'logprobs': None}, id='run-f0d078eb-2c6d-44ad-9754-47deddf45ff6-0', usage_metadata={'input_tokens': 127, 'output_tokens': 8, 'total_tokens': 135})

In [30]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [32]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Subhasish")],"language":"Bengali"})
response.content

'নমস্কার, আমি আপনার সাহায্যকারী! আমি আপনার নাম শুনেছি, সুভাষিষ! আমি সাহায্য করতে চাই, আপনার যা কিছু প্রশ্ন আছে বা সমস্যা আছে, তা আমি ব্যাপারে সাহায্য করব।'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [33]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")

In [35]:
config={"configurable":{"session_id":"Chat-1"}}
response=with_message_history.invoke(
    {'messages': [HumanMessage(content="what is my name?")],"language":"Bengali"},
    config=config
)
response.content

'আপনার নাম সুব্রতিশ!'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [44]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer=trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

In [50]:
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm Subhasish"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content="hi! I'm Subhasish"),
 AIMessage(content='hi!'),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [51]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

'vanilla ice cream'

In [52]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"Chat-1"}}

In [53]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

'Subhasish'